# 오버피팅과 언더피팅

- 오버피팅(overfitting)
> 과도하게 적합, 새로운 데이터셋에 적용하기 어려움, 실제 모형보다 큰 차원으로 적합, 복잡한 모형일 수록 가능성 높음, 분산 커짐
- 언더피팅(underfitting)
> 데이터의 특성 잘 나타내지 못함, 실제 모형보다 작은 차원으로 적합, 간단한 모형일 수록 가능성 높음, 편향 큰 현상 나타남

# Cross-validation (교차검증)

- overfitting과 underfitting을 방지하고 적합한 모형 추정하기 위한 방법
- 정답데이터를 학습데이터와 평가데이터로 나누고, 학습데이터의 일부를 검증데이터로 사용
- 학습데이터: parameter 정하기 위해, 검증데이터: hyperparameter 정하기 위해 사용
- hyperparameter 결정하는데 학습/평가데이터만 존재한다면 평가데이터에 의해 하이퍼파라미터 결정되니까 안됨 (평가데이터는 오직 모형의 성능을 평가하는 목적으로만 사용)

- k-fold cross validation...

# 파이프라인

- 데이터 전처리(스케일링 등)와 학습 모형을 연결해 코드를 간결화할 수 있음

##### 파이프라인 적용하기 전 학습과정

표준화 스케일링>>

std_scale = StandardScaler()

X_tn_std = std_scale.fit_transform(x_tn)

X_te_std = std_scale.transform(X_te)


학습>>

clf_linear = LinearRegression

clf_linear.fit(X_tn_std, y_tn)

##### 파이프라인 적용한 코드

파이프라인>>

linear_pipline = Pipeline([

    ('scaler', StandardScaler()),
    
    ('linear_regression', LinearRegression())
    
])

학습>>

linear_pipline.fit(X_tn, y_tn)

In [4]:
# 파이프라인 사용 전 전체 코드
from sklearn import datasets
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

raw_boston = datasets.load_boston()

X = raw_boston.data
y = raw_boston.target

# 데이터 분할
X_tn, X_te, y_tn, y_te = train_test_split(X, y, random_state=7)

# 표준화 스케일링
std_scale = StandardScaler()
X_tn_std = std_scale.fit_transform(X_tn)
X_te_std = std_scale.transform(X_te)

# 학습
clf_linear = LinearRegression()
clf_linear.fit(X_tn_std, y_tn)

# 예측
pred_linear = clf_linear.predict(X_te_std)

# 평가
mean_squared_error(y_te, pred_linear)

29.515137790197596

In [6]:
# 파이프라인 사용 후 전체 코드

# 데이터 분할
X_tn, X_te, y_tn, y_te = train_test_split(X, y, random_state=7)

# 파이프라인
linear_pipline = Pipeline([
    ('scaler', StandardScaler()),
    ('linear_regression', LinearRegression())
])

# 학습
linear_pipline.fit(X_tn, y_tn)

# 예측
pred_linear = linear_pipline.predict(X_te)

# 평가
mean_squared_error(y_te, pred_linear)

29.515137790197596

# 그리드서치 (grid search)

- 관심 있는 매개변수들을 대상으로 학습 가능하도록 만드는 방식

In [8]:
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split

from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

# 꽃 데이터 불러오기
raw_iris = datasets.load_iris()

# 피처/타깃
X = raw_iris.data
y = raw_iris.target

# 데이터 분할
X_tn, X_te, y_tn, y_te = train_test_split(X, y, random_state=0)

# 표준화 스케일링
std_scale = StandardScaler()
std_scale.fit(X_tn)
X_tn_std = std_scale.transform(X_tn)
X_te_std = std_scale.transform(X_te)

# 그리드 서치
best_accuracy = 0 # 가장 높은 정확도 나타내는 best_accuracy라는 변수 초기화
 
for k in [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]: # 관심있는 하이퍼파라미터의 범위 정해 반복문 수행
    clf_knn = KNeighborsClassifier(n_neighbors=k) # 해당 k값 적용한 KNN 실행
    clf_knn.fit(X_tn_std, y_tn) # 학습
    knn_pred = clf_knn.predict(X_te_std) # 예측
    accuracy = accuracy_score(y_te, knn_pred) # 평가(정확도)
    if accuracy > best_accuracy:
        best_accuracy = accuracy # best_accuracy 갱신
        final_k = {'k': k}
        
print(final_k)
print(accuracy)

{'k': 3}
0.9736842105263158


# 손실 함수와 비용 함수

### 손실함수와 비용함수의 개념

- 손실함수 (loss function)
> - 생성한 모형이 실젯값과 얼마나 차이가 나는지 (손실 정도를 수치로 나타냄)
> - 각 데이터 포인트에 대해 차이 나타냄
- 비용함수 (cost function)
> - 전체 데이터셋을 대상으로 하는 손실 나타냄

#### L1 손실함수 (Lasso)

- 실젯값과 예측값 차이에 기댓값을 취한 것
- 실젯값과 예측값의 차이를 줄이는 것이 학습 목적
- 관련된 비용함수: MAE (이상치와 상관 없이 안정적)

#### L2 손실함수 (Ridge)

- 실젯값과 예측값의 차이에 제곱을 취한 것
- 관련된 비용함수: MSE (이상치에 민감), RMSE (이상치와 상관없이 안정적)

### 엔트로피 (entropy)
손실함수

#### 엔트로피 (Entropy)
- 정보이론에서 사용
- 확률 변수의 불확실성 정도를 측정하기 위해 사용
- 의사 결정 나무에서 주로 사용
- 하나의 분포를 대상으로

#### 크로스 엔트로피 (Cross Entropy)
- 두 분포를 대상으로 엔트로피를 측정해 두 분포 간의 차이 계산
- P(x): 실제 모형의 분포, Q(x): 추정 모형의 분포

#### Kullback-Leibler Divergence (KLD)
- 상대적 엔트로피(relative entropy)

### Negative Log Likelihood (NLL)
손실함수

로그 가능도 함수를 최대화 하는 것은 크로스-엔트로피를 최소화하는 것과 같음
-> 로그 가능도 함수르르 최대화 하는 파라미터를 찾는 문제는 CE를 최소화하는 파라미터를 찾는 문제와 동일하므로 NLL을 손실함수로 사용할 수 있음

# 모형 성능 평가

### 모형 성능 평가에 필요한 개념

- 정밀도(Precision), 리콜(Recall)=민감도(Sensitivity), 특이도(Specificity), FPR, 정확도(accuracy), 에러율(Error Rate), ROC 커브(Receiver Operating Characteristic)

### 분류 문제에서의 성능평가 (Classification Problem)

- 지시함수(indicator function)

In [9]:
# 정확도 측정하기 위해 accuracy_score 함수 사용 (normalize=True가 디폴트, 1에 가까울 수록 정확도 높다는 의미)
from sklearn.metrics import accuracy_score

y_pred = [0, 2, 1, 3]
y_true = [0, 1, 2, 3]

# 정확도
print(accuracy_score(y_true, y_pred))
# normalize=False 사용하면 0부터 1 사잇값이 아닌 예측값과 실젯값과 일치하는 빈도수 출력
print(accuracy_score(y_true, y_pred, normalize=False))

0.5
2


##### F1 score
- precision과 recall의 조화 평균
- 0부터 1 사이 값을 가지며, 1에 가까울수록 높은 성능 나타냄

##### Confusion Matrix

In [10]:
from sklearn.metrics import confusion_matrix
y_true = [2, 0, 2, 2, 0, 1]
y_pred = [0, 0, 2, 2, 0, 2]
confusion_matrix(y_true, y_pred)

array([[2, 0, 0],
       [0, 0, 1],
       [1, 0, 2]], dtype=int64)

위에서 부터 차례로 클래스 0, 1, 2를 의미하고 행렬의 행은 실젯값을 의미하며 열은 예측값을 의미함

##### Classification Report

여러 성능 점수를 한 번에 확인할 수 있음

- accuracy: 전체 정확도
- macro avg: 라벨별 가중치 부여하지 않은 평균값
- weighted avg: support-weighted된 평균값
- support: 실젯값(y_true)의 클래스별 데이터 개수

In [12]:
from sklearn.metrics import classification_report
y_true = [0, 1, 2, 2, 0]
y_pred = [0, 0, 2, 1, 0]
target_names = ['class 0', 'class 1', 'class 2']
print(classification_report(y_true, y_pred, target_names=target_names))

              precision    recall  f1-score   support

     class 0       0.67      1.00      0.80         2
     class 1       0.00      0.00      0.00         1
     class 2       1.00      0.50      0.67         2

    accuracy                           0.60         5
   macro avg       0.56      0.50      0.49         5
weighted avg       0.67      0.60      0.59         5



### 회귀 문제에서의 성능 평가 (Regression)

##### Mean Absolute Error
- 예측값과 실젯값의 차이의 절댓값의 평균

In [15]:
from sklearn.metrics import mean_absolute_error
y_true = [3, -0.5, 2, 7]
y_pred = [2.5, 0.0, 2, 8]
print(mean_absolute_error(y_true, y_pred))

0.5


##### Mean Squared Error
- 오차 제곱합. 오차의 제곱합의 평균

In [16]:
from sklearn.metrics import mean_squared_error
y_true = [3, -0.5, 2, 7]
y_pred = [2.5, 0.0, 2, 8]
print(mean_squared_error(y_true, y_pred))

0.375


##### R2 score

In [17]:
from sklearn.metrics import r2_score
y_true = [3, -0.5, 2, 7]
y_pred = [2.5, 0.0, 2, 8]
print(r2_score(y_true, y_pred))

0.9486081370449679


### 군집 문제에서의 성능 평가 (clustering)
- 비지도학습

##### 실루엣스코어 (silhouette score)
- 서로 다른 군집이 얼마나 잘 분리되는지를 나타내는 지표
- 같은 군집의 데이터는 가까운 거리에 뭉쳐 있고, 다른 군집의 데이터끼리는 멀리 떨어져 있을 수록 높은 점수

In [18]:
from sklearn.metrics import silhouette_score
X = [[1, 2], [4, 5], [2, 1], [6, 7], [2, 3]] # 피처 2개로 구성된 데이터 5개
labels = [0, 1, 0, 1, 0] # 해당 피처의 클래스 설정
sil_score = silhouette_score(X, labels)
print(sil_score)

0.5789497702625118
